to do:
- image in tensorboard (test alpha mobilenet)
- imports aufräumen
- Secondstage.py
- Konfiguration außerhalb:
    - Name output
    - cat/ bin/ reg
- endliche Anzahl an Runden

In [2]:
%tensorflow_version 1.x
!pip install tensorflow==1.14

!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

from google.colab import drive, files
drive.mount('/content/gdrive')

     |████████████████████████████████| 109.2MB 28kB/s 
     |████████████████████████████████| 491kB 38.0MB/s 
     |████████████████████████████████| 3.2MB 39.6MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
Selecting previously unselected package python-bs4.
(Reading database ... 145155 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Sele

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:

from tqdm import tqdm
import re
from datetime import datetime
from multiprocessing import Process, Queue

from copy import deepcopy

from keras import applications, optimizers, backend as K
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Reshape, Conv2D, Activation
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, Callback, LambdaCallback
from keras.utils import np_utils
import pandas as pd
import numpy as np
from numpy.random import randint
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import io
import sys
import csv, json, pickle
from lxml import etree


os.chdir('/content/gdrive/My Drive/')
from SecondStage_colab import second_stage_utils
from SecondStage_colab import file_utils

sys.path.append('/content/gdrive/My Drive/SecondStage_colab/')
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/SecondStage_colab/'

Using TensorFlow backend.


In [0]:
#NAME =
#example: cat_exptype_dataset(training)_dataset(eval)_conf(batch size etc) 

LOG_PATH = '/content/gdrive/My Drive/SecondStage_colab/output/'
LABEL_MAP_PATH = '/content/gdrive/My Drive/SecondStage_colab/label_map.pbtxt'
TRAIN_RECORD = '/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot9_13colors.record'
EVAL_RECORD = '/content/gdrive/My Drive/data/generated/SecondStage_X/validation_rot3_13colors.record'

GPU = True

In [0]:
from file_utils import save_json
from second_stage_utils import *

def train(train_record, conf, out, rep=1):
    timestamp = "{:%Y-%m-%d-%H-%M}".format(datetime.now())
    log_path = LOG_PATH + conf['name'] + '_' + out + '/' + timestamp + '-r' + str(rep) + '/'
    os.makedirs(log_path, exist_ok=True)
    save_json(log_path + '/experiment_config.json', conf)

    label_map = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)

    num_classes = label_map_util.get_max_label_map_index(label_map_util.load_labelmap(LABEL_MAP_PATH)) + 1

    X,Y,Z,_ = tf_record_load_crops([train_record])

    X_train, Y_train, Z_train = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_train = angle_to_bin(Z_train)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")
    print(Y_train.shape)
    print(num_classes)
    print(Z2_train.shape)
    print(NUM_ORI_BINS)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")

    eval_records = EVAL_RECORD

    X,Y,Z,_ = tf_record_extract_crops([eval_records], 1, 0.0, 0.0)
    X_val, Y_val, Z_val = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_val = angle_to_bin(Z_val)

    assert len(X_val) > 0 and len(Y_val) > 0 and len(Z_val) > 0, '{} is incomplete'.format(eval_records)

    outputs = None
    model_final = None
    summary = TensorBoardCustom(log_dir=log_path, label_map=label_map, AddCustomMetrics=(out == '_cat'))

    filepath=log_path+"model-{epoch:02d}.h5"
    checkpoint = ModelCheckpoint(filepath, verbose=1, period=20)
    if conf['optimizer'] == 'rmsprop':
        optimizer = optimizers.RMSprop(lr=conf['learning_rate'])
    elif conf['optimizer'] == 'adam':
        optimizer = optimizers.Adam(lr=conf['learning_rate'])
    else:
        raise Error('Unknown optimizer: ' + conf['optimizer'])


    mobilenet_base = applications.mobilenet.MobileNet(alpha = conf['alpha'],
                                                      weights = "imagenet",
                                                      include_top=False,
                                                      dropout = conf['dropout'],
                                                      input_shape = (
                                                      conf['img_size'],
                                                      conf['img_size'],
                                                        3
                                                      ))
    shape = (1, 1, int(1024 * conf['alpha']))
    x = GlobalAveragePooling2D()(mobilenet_base.output)
    x = Reshape(shape, name='reshape_1')(x)
    #x = Dropout(conf['dropout'], name='dropout')(x)

    if out == '_cat':
      # Branch classification
      cat = Conv2D(num_classes, (1, 1),
                padding='same', name='conv_cat')(x)
      cat = Activation('softmax', name='act_softmax')(cat)
      cat = Reshape((num_classes,), name='cat_out')(cat)
      outputs = cat
      model_final = Model(inputs = mobilenet_base.input, outputs = outputs)
      model_final.compile(optimizer = optimizer,
                          loss={'cat_out': 'categorical_crossentropy'},
                          metrics ={'cat_out': 'accuracy'})
      model_final.fit(
        X_train,
        {'cat_out': Y_train},
        validation_data=(X_val, Y_val),
        batch_size=conf['batch_size'], epochs = conf['epochs_cat'], verbose=1,
        callbacks=[summary,checkpoint],
        shuffle=True
      )
      
    elif out == '_reg':
      # Branch regression
      reg = Conv2D(1, (1, 1),
                padding='same', name='conv_reg')(x)
      reg = Activation('linear', name='act_linear')(reg)
      reg = Reshape((1,), name='reg_out')(reg)
      outputs = reg
      model_final = Model(inputs = mobilenet_base.input, outputs = outputs)
      model_final.compile(optimizer = optimizer,
                          loss={'reg_out': angle_mse},
                          metrics ={'reg_out': angle_mae})
      model_final.fit(
        X_train,
        {'reg_out': Z_train},
        validation_data=(X_val, Z_val),
        batch_size=conf['batch_size'], epochs = conf['epochs_reg'], verbose=1,
        callbacks=[summary,checkpoint],
        shuffle=True
      )
            
    elif out == '_bin':
      # Branch orientation classification with bins
      bin = Conv2D(NUM_ORI_BINS, (1, 1),
                padding='same', name='conv_bin')(x)
      bin = Activation('softmax', name='act_bin')(bin)
      bin = Reshape((NUM_ORI_BINS,), name='bin_out')(bin)
      outputs = bin
      model_final = Model(inputs = mobilenet_base.input, outputs = outputs)
      model_final.compile(optimizer = optimizer,
                          loss= {'bin_out': 'categorical_crossentropy'},
                          metrics = [angle_bin_mae, angle_bin_rmse])
      model_final.fit(
        X_train,
        {'bin_out': Z2_train},
        validation_data=(X_val, Z2_val),
        batch_size=conf['batch_size'], epochs = conf['epochs_bin'], verbose=1,
        callbacks=[summary,checkpoint],
        shuffle=True
      )
    else:
        print('UNKNOWN OUTPUT CONFIG {}'.format(out))

    model_final.save(log_path+"model-final.h5")
    print(log_path)
    print("Finished training for {}".format(conf['name']))


In [0]:
default_sstage_conf = {
    'dataset': 'default',
    'epochs_cat': 40,
    'epochs_reg': 40,
    'epochs_bin': 40,
    'optimizer': 'adam',
    'learning_rate': 3e-4,
    'dropout': 0.001,
    'alpha': 0.5,
    'img_size': 35,
    'repetions': 1,
    'batch_size': 7020,
}

def create_all_sstage_experiments():
    configs = []
    #configs.extend(create_sstage_default())
    #configs.extend(create_sstage_dropouts())
    #configs.extend(create_sstage_alphas())
    #configs.extend(create_sstage_dropouts_alphas())
    configs.extend(create_sstage_batch_sizes())
    return configs

def create_sstage_batch_sizes():
    config = []
    batchsizes = [10530, 7020, 3510, 2106, 1053]
    for size in batchsizes:
      modified = deepcopy(default_sstage_conf)
      modified['name'] = "sstage_default_bsize" + str(size)
      modified['batch_size'] = size
      config.append(deepcopy(modified))
    return config

def create_sstage_dropouts_alphas():
    config = []
    dropouts = [0.001, 0.3, 0.4, 0.5, 0.6] #
    alphas = [0.25] # 0.5, 0.75, 1.0
    for drop in dropouts:
      modified = deepcopy(default_sstage_conf)
      modified['name'] = "ssdef_drop" + str(drop)
      modified['dropout'] = drop
      for alp in alphas:
        modifiedB = deepcopy(modified)
        modifiedB['name'] = modifiedB['name'] + "_alpha" + str(alp)
        modifiedB['alpha'] = alp
        config.append(deepcopy(modifiedB))
    return config

def create_sstage_dropouts():
    config = []
    dropouts = [0.25]
    for drop in dropouts:
      modified = deepcopy(default_sstage_conf)
      modified['name'] = "sstage_default_drop" + str(drop)
      modified['dropout'] = drop
      config.append(deepcopy(modified))
    return config

def create_sstage_alphas():
    config = []
    alphas = [0.5, 0.75, 1.0]
    for alp in alphas:
      modified = deepcopy(default_sstage_conf)
      modified['name'] = "sstage_default_alpha" + str(alp)
      modified['alpha'] = alp
      config.append(deepcopy(modified))
    return config

def create_sstage_default():
    config = []
    modified = deepcopy(default_sstage_conf)
    modified['name'] = "sstage_default"
    config.append(deepcopy(modified))
    return config

exp = create_all_sstage_experiments()
TORUN = []
TORUN.append('_bin')
TORUN.append('_cat')
train_records = [ 'training_rot9_13colors.record']
for config in tqdm(exp):
    print("or_conf", config)
    for r in range(config['repetions']):     
        for out in TORUN: # 
            for train_record in train_records:
              conf = deepcopy(config)
              conf['name'] = conf['name']+'_'+out+"_"+train_record
              print("Start training:  " + conf['name'])
              
              if not GPU:
                  p = Process(
                    target=train,
                    args=("/content/gdrive/My Drive/data/generated/SecondStage_X/" + train_record, conf, out, r)
                  )
                  p.start() 
              else:
                train("/content/gdrive/My Drive/data/generated/SecondStage_X/" + train_record, conf, out)

  0%|          | 0/5 [00:00<?, ?it/s]

or_conf {'dataset': 'default', 'epochs_cat': 40, 'epochs_reg': 40, 'epochs_bin': 40, 'optimizer': 'adam', 'learning_rate': 0.0003, 'dropout': 0.001, 'alpha': 0.5, 'img_size': 35, 'repetions': 1, 'batch_size': 10530, 'name': 'sstage_default_bsize10530'}
Start training:  sstage_default_bsize10530__bin_training_rot9_13colors.record

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded 210600 crops from ['/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot9_13colors.record']
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´
(210600, 14)
14
(210600, 360)
360
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´








/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


5578752/5577668 [==============================] - 0s 0us/step

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 210600 samples, validate on 27000 samples


Epoch 1/40
210600/210600 [==============================] - 517s 2ms/step - loss: 6.8756 - angle_bin_mae: 87.3739 - angle_bin_rmse: 103.3843 - val_loss: 7.6321 - val_angle_bin_mae: 82.0873 - val_angle_bin_rmse: 101.9405
Epoch 2/40
 52650/210600 [======>.......................] - ETA: 6:11 - loss: 6.0234 - angle_bin_mae: 82.7929 - angle_bin_rmse: 102.3790

In [0]:
%load_ext tensorboard
%reload_ext tensorboard
# LOG_PATH did not work for some reason
%tensorboard --logdir '/content/gdrive/My Drive/SecondStage_colab/output/'

In [0]:
#!pip install -U tensorboard >piplog 2>&1
#!tensorboard dev upload --logdir '/content/gdrive/My Drive/SecondStage_colab/output/'

In [0]:
!pip install -U tensorboard >piplog 2>&1
!tensorboard dev list
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE

In [0]:
#!rm "/content/gdrive/My Drive/SecondStage_colab/output/" -rf
#!ls "/content/gdrive/My Drive/SecondStage_colab/"